In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import momepy as mm
import networkx as nx
from shapely import Point, LineString, reverse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from pyproj import Transformer
import datetime

c:\Users\ruben\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. LECTURA DELS NODES I DELS ARCS. CREACIÓ DEL GRAF

In [2]:
# Introduim la xarxa completa en un Graph de NetworkX
node = gpd.read_file("data/networks/AdG_node.gpkg")
arc = gpd.read_file("data/networks/AdG_arc.gpkg")


G0 = nx.DiGraph()
G0.add_nodes_from([(row['node_id'], row) for index, row in node.iterrows() if row['state']!=0])
G0.add_edges_from([(row['node_1'], row['node_2'], row) for index, row in arc.iterrows() if row['state']!=0])
# estem eliminant els obsolets escollint nomes els state=0 (OPERATIVO)

print('La xarxa OPERATIVA completa conte: ')
print('nodes = ', len(G0.nodes))
print('arcs =', len(G0.edges))

La xarxa OPERATIVA completa conte: 
nodes =  13572
arcs = 13682


In [48]:
# Read node data from CSV file
node_data = pd.read_csv("data/node2021.csv")

# Create a dictionary to convert the node data to a format that can be used with NetworkX
node_dict = {}
for index, row in node_data.iterrows():
    if row['state'] != 0: # Check if the node is not obsolete
        node_id = row['node_id']
        node_data = row.to_dict()
        node_dict[node_id] = node_data

# Read edge data from CSV file
edge_data = pd.read_csv("data/arc2021.csv")

# Create a list of tuples containing the edge data
edge_list = []
for index, row in edge_data.iterrows():
    if row['state'] != 0: # Check if the edge is not obsolete
        node_1 = row['node_1']
        node_2 = row['node_2']
        edge_data = row.to_dict()
        edge_list.append((node_1, node_2, edge_data))

# Create a NetworkX graph and add the nodes and edges
G21 = nx.DiGraph()
G21.add_nodes_from(node_dict.items())
G21.add_edges_from(edge_list)

print('La xarxa OPERATIVA completa conte: ')
print('nodes = ', len(G21.nodes))
print('arcs =', len(G21.edges))

La xarxa OPERATIVA completa conte: 
nodes =  13572
arcs = 13682


In [42]:
# Read node data from CSV file
node_data = pd.read_csv("data/node2023.csv")

# Create a dictionary to convert the node data to a format that can be used with NetworkX
node_dict = {}
for index, row in node_data.iterrows():
    if row['state'] != 0: # Check if the node is not obsolete
        node_id = row['node_id']
        node_data = row.to_dict()
        node_dict[node_id] = node_data

# Read edge data from CSV file
edge_data = pd.read_csv("data/arc2023.csv")

# Create a list of tuples containing the edge data
edge_list = []
for index, row in edge_data.iterrows():
    if row['state'] != 0: # Check if the edge is not obsolete
        node_1 = row['node_1']
        node_2 = row['node_2']
        edge_data = row.to_dict()
        edge_list.append((node_1, node_2, edge_data))

# Create a NetworkX graph and add the nodes and edges
G23 = nx.DiGraph()
G23.add_nodes_from(node_dict.items())
G23.add_edges_from(edge_list)

print('La xarxa OPERATIVA completa conte: ')
print('nodes = ', len(G23.nodes))
print('arcs =', len(G23.edges))

C:\Users\ruben\AppData\Local\Temp\ipykernel_15992\3617289775.py:13: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  edge_data = pd.read_csv("data/arc2023.csv")


La xarxa OPERATIVA completa conte: 
nodes =  13608
arcs = 13720


Ens inventem els codis d'aquells que no en tenen un de correcte.

In [176]:
nodes_amb_codi_erroni =([node for node, data in G.nodes(data=True) if data['code'][0:2] != 'PR'])

i = 0
for node in nodes_amb_codi_erroni:
  data = G.nodes[node]
  data['code'] = 'DESC' + str(i)
  i += 1

In [43]:
nodes_amb_codi_erroni_21 =([node for node, data in G21.nodes(data=True) if data['code'][0:2] != 'PR'])
nodes_amb_codi_erroni_23 =([node for node, data in G23.nodes(data=True) if data['code'][0:2] != 'PR'])
print('nodes_amb_codi_erroni pel 2021 =', len(nodes_amb_codi_erroni_21) )
print('nodes_amb_codi_erroni pel 2023 -', len(nodes_amb_codi_erroni_23) )

nodes_amb_codi_erroni pel 2021 = 2334
nodes_amb_codi_erroni pel 2023 - 2346


## 2. CREACIÓ DEL SUBGRAF QUE CONTÉ LA DEPURADORA

Definim un subGraph com la xarxa connectada a la depuradora.

no hi ha cap node/arc desconectat en el subGraph

estem extraien tambe els sistemes de drenatge urba no connectats a la xarxa residual. 

suposem que no existeixen nodes sanitaris fora d'aquesta xarxa (cal comprovar).

obenim una xarxa perfectament connectada.

També es detecta els nodes que no tenen sortida i no són un outfall.

In [49]:
#Extraiem la sub-xarxa connectada a la depuradora
#Definim la depuradora

G=G21
edar_code= 'PR0005300'
edar=[node for node, data in G.nodes(data=True) if data['code'] == edar_code][0]

# definim el subGraph associat:
G_notDi = nx.Graph(G)
subGraph = nx.DiGraph(nx.subgraph(G,nx.node_connected_component(G_notDi, edar)))

print('La xarxa connectada a la depuradora conte: ')
print( 'nodes = ', len(subGraph.nodes))
print('arcs =', len(subGraph.edges))
print('---')


# nodes marcats com OUTFALL
outfalls= [node for node, data in subGraph.nodes(data=True) if data['epa_type'] == 'OUTFALL']
print('nodes OUTFALL =', len(outfalls))

# nodes marcats com CHAMBERS
chambers_netinit= [node for node, data in subGraph.nodes(data=True) if data['nodecat_id'] == 'CEC' and subGraph.degree(node)==1]
print('nodes CHAMBER-NETINIT =', len(chambers_netinit))

# detectem els nodes tipo sink del subGraph
sink = {node: data for node, data in subGraph.nodes(data=True) if subGraph.out_degree(node) == 0 and node!=edar and data['epa_type'] != 'OUTFALL'}
print('nodes tipo sink =', len(sink))

# arcs marcats com Sobreeixidor
sobreeixidors = [(node1, node2) for node1, node2, data in subGraph.edges(data=True) if data['category_type'] == 'Sobreeixidor']
print('arcs sobreeixidor =', len(sobreeixidors))


La xarxa connectada a la depuradora conte: 
nodes =  12017
arcs = 12516
---
nodes OUTFALL = 128
nodes CHAMBER-NETINIT = 15
nodes tipo sink = 37
arcs sobreeixidor = 277


In [32]:
chambers_netinit

[11495,
 12305,
 12818,
 14941,
 15596,
 17563,
 1896,
 2109,
 42936,
 4869,
 5162,
 7387,
 7826,
 8267,
 9452]

Revisem els nodes sinks i eliminem aquells que no son Chambers. S'eliminen alguns que son bons (2003, 33882, 34892, 6242, 9589, node_id d'adg21), pero la majoria no tenen cap informacio.

No afecten gaire al resultat perque son nodes inici de red que creiem que tenen l'aresta girada.

In [50]:
bad_sink = [key for key, value in sink.items() if subGraph.degree(key) == 1 and value['nodecat_id'] !='CEC']

subGraph.remove_nodes_from(bad_sink)
print('La xarxa connectada a la depuradora conte: ')
print( 'nodes = ', len(subGraph.nodes))
print('arcs =', len(subGraph.edges))
print('---')

La xarxa connectada a la depuradora conte: 
nodes =  11986
arcs = 12485
---


In [25]:
set(sink) - set(bad_sink)

{3670, 12220, 12818, 30191, 45423, 51236}

In [28]:
no_llegan_depuradora = [node for node in subGraph.nodes if not nx.has_path(subGraph, node, edar)]
len(no_llegan_depuradora)

860

direccions
1. sub-xarxa connectada a la depuradora : subGraph
2. sink: inicis de xarxa incorrectes, corregir direccions. report changes
3. comprovar direccions edge: tots aquells arcs que no son sistemes de sobreiximent, han danar direxionats a la depuradora.


nodes
1.1 alguns nodes tenen code incorrecte, comunicar reportar casos, mantenim el node_id de adg code=PRdnode_id
1.2 top_elev: corregir nan amb z, comprovar valors fora del rang esperat z-1 < top_elev < z+1
   colocar valors a custom_top_elev
1.3 ymax: corregim nan amb '1'. colocar canvis a custom_ymax
1.4 matcat_id if null write DESC


In [11]:
# 1.1 corregim el code dels nodes amb identificadoe incorrectes

nodes_amb_codi_erroni =([node for node, data in subGraph.nodes(data=True) if data['code'][0:2] != 'PR'])

i = 0
for node in nodes_amb_codi_erroni:
  data = subGraph.nodes[node]
  data['code'] = 'PRd' + str(data['node_id'] )
  i += 1


In [ ]:
for index, row in subGraph.nodes(data=True):
    if row['state'] != 0:
        x = row['geometry'].x
        y = row['geometry'].y


In [89]:
# 1.2 corregim top_elev
#utilitzem el graph G:
G=G21


# per als <0 (NULL included)
nodes_amb_top_elev_empty =([node for node, data in G.nodes(data=True) if float(data['top_elev']) >= 0])
print('nodes_amb_top_elev_empt = ', len(nodes_amb_top_elev_empty))

#corregim els valors a custom_top_elev
i = 0
for node in nodes_amb_top_elev_empty:
  data = G.nodes[node]
  data['custom_top_elev'] = data['geometry'].z   #com puc accedir a la z ??
  i += 1

print('despres de corregir...')
nodes_amb_top_elev_empty =([node for node, data in G.nodes(data=True) if (float(data['top_elev']) >= 0 or float(data['custom_top_elev']) >= 0) ]  )
print('nodes_amb_top_elev_empt = ',len(nodes_amb_top_elev_empty))
  


# comprovem que top_elev es trpba en el rang esperat z-1<top_elev<z+1
masc_top_elev=  (float(row['geometry'].z)-1) < float(data['top_elev']) < (float(row['geometry'].z)+1)
nodes_amb_top_elev_erroni =([node for node, data in G.nodes(data=True) if masc_top_elev])


#print(len(nodes_amb_top_elev_erroni))

nodes_amb_top_elev_empt =  8267


KeyError: 'geometry'

In [58]:
my_node=[node for node, data in G21.nodes(data=True) if data['node_id'] == '39079']
print(my_node)

my_arc=[(node1, node2) for node1, node2, data in subGraph.edges(data=True) if data['arc_id'] == '2362']
#print(my_arc)

[]


nodes

1. top_elev: corregir nan amb z, comprovar valors fora del rang esperat z-1 < top_elev < z+1
   colocar valors a custom_top_elev
2. ymax: corregim nan amb '1'. colocar canvis a custom_ymax
4. matcat_id if null write DESC

nota: alguns nodes tenen code incorrecte, comunicar reportar casos, mantenim el node_id de adg code=PRDnode_id

arcs

1. comprovar node_1/2, alguns valors em surten null* a les taules, com pot ser?
2. comprovar null* en nodetype_1/2
3. y1: comprovar valors fora del rang esperat (0,10), corregir valor superiors a 10 , if nan* posar y1=ymax(node1)
*giswater hauria de corregir els nan si corregim els ymax dels nodes
   comprovar que: ymax(node1)-1 < y1 < ymax(node1)+1
   colocar valors corregits a costum_y1

## 3. CORRECCIÓ DE LES DIRECCIONS

definim node de "sobreximent" com nodes direccionats cap a un node que no es la depuradura
    
volem que els nodes de sobreiximent detectats coiniceixin amb els catalogats com a outfall o chamber-netinit 

considerem errors de diseny les diferencies -> corregim

#TODO afegir tambe aquelles chambers sense predecesor (netinit) a la llista de permesos
    
#TODO imprimir llista de diferencies per comunicar a AdG

In [52]:
# TODO: problemes amb els nodes tipus "CHAMBER" i "STORAGE". Crear un nou arc?
# S'intercanvien els valors del node 1 i node 2 de l'aresta
# 
attributes = ['node_', 'y', 'custom_y', 'elev', 'custom_elev', 'sys_elev', 'nodetype_', 'sys_y', 'r', 'z']
        
def intercambiarValores(data):
  
  for attribute in attributes:
    if attribute+'1' in data:
      aux = data[attribute+'1']
      data[attribute+'1'] = data[attribute+'2']
      data[attribute+'2'] = aux
    
  if 'geometry' in data:
    data['geometry'] = reverse(data['geometry'])
  
  return data


# Per direccionar els arcs del subGraph que no arriben a la depuradora o als nodes de sobreiximent
# 1. Trobem tots els nodes que no arriben a la depuradora
# 2. Trobem els nodes del punt 1 que no arriben a cap outfall
no_llegan_depuradora = [node for node in subGraph.nodes if not nx.has_path(subGraph, node, edar)]

no_llegan_outfall = []
for node in no_llegan_depuradora:
    outfalls_i_chambers = outfalls + chambers_netinit
    if node not in outfalls_i_chambers :
      llega = False
      i = 0
      while not llega and len(outfalls_i_chambers)>i:
        llega = nx.has_path(subGraph, node, outfalls_i_chambers[i])
        i += 1
      if not llega:
        no_llegan_outfall.append(node)

print('nodes que no arriben a la depuradora =', len(no_llegan_depuradora))
print('nodes que no arriben a la depuradora ni a un outfall ni a un chamber =', len(no_llegan_outfall))

malament = no_llegan_outfall.copy()

# 3. Mentre hi hagi nodes que no arribin a la depuradora ni a un outfall
# 3.1 Per tots aquests nodes
# 3.1.1 Mirem si els seus veins arriben a la depuradora
# 3.1.2 Si algun vei arriba a la depuradora, afegim un nou arc que va del node al vei i eliminem l'antic
# 3.2 Si cap dels seus veins arriba a la depuradora, s'afegeix de nou a la llista  de nodes a revisar
arcs_girats = []
while len(no_llegan_outfall)>0:
  no = []
  for node in no_llegan_outfall:
    teCami = nx.has_path(subGraph, node, edar)
    if not teCami:
      veins = list(nx.all_neighbors(subGraph, node))
    while not teCami and len(veins)>0:
      v = veins.pop()
      teCami = nx.has_path(subGraph, v, edar)
      if teCami and subGraph.has_edge(v,node):
        data = intercambiarValores(subGraph.get_edge_data(v, node))
        data['girada'] = True
        subGraph.add_edge(node, v, **data)
        subGraph.remove_edge(v, node)
        arcs_girats.append(data['code'])
    if not teCami:
      no.append(node)
  no_llegan_outfall = no
  
# Si tornem a veure els nodes que no arriben a la depuradora, no haurien de quedar cap

no_llegan_depuradora_arreglat = [node for node in subGraph.nodes if not nx.has_path(subGraph, node, edar)]

no_llegan_outfall_arreglat = []
for node in no_llegan_depuradora_arreglat:
    if node not in outfalls:
      llega = False
      i = 0
      while not llega and len(outfalls)>i:
        llega = nx.has_path(subGraph, node, outfalls[i])
        i += 1
      if not llega:
        no_llegan_outfall_arreglat.append(node)
        
print('nodes que no arriben a la depuradora després del direccionament =', len(no_llegan_depuradora_arreglat))
print('nodes que no arriben a la depuradora ni a un outfall després del direccionament =', len(no_llegan_outfall_arreglat))

nodes que no arriben a la depuradora = 860
nodes que no arriben a la depuradora ni a un outfall ni a un chamber = 38
nodes que no arriben a la depuradora després del direccionament = 822
nodes que no arriben a la depuradora ni a un outfall després del direccionament = 1


In [53]:
no_llegan_outfall_arreglat


[12818]

In [38]:
subGraph.get_edge_data(v, node)

{'arc_id': 12217,
 'code': 'TR0023709',
 'node_1': 12220.0,
 'node_2': 12216.0,
 'y1': nan,
 'y2': nan,
 'elev1': nan,
 'elev2': nan,
 'custom_y1': nan,
 'custom_y2': nan,
 'custom_elev1': nan,
 'custom_elev2': nan,
 'sys_elev1': nan,
 'sys_elev2': nan,
 'arc_type': 'TRAM',
 'arccat_id': 'PVC160-CC',
 'matcat_id': nan,
 'epa_type': 'CONDUIT',
 'sector_id': 1,
 'state': 1,
 'state_type': 2,
 'annotation': 'SE15_75',
 'observ': 'PRIVAT-TRAM RECULL ESC.-O. ESTUDI GIRONA',
 'comment': nan,
 'sys_slope': nan,
 'inverted_slope': nan,
 'custom_length': nan,
 'dma_id': 0,
 'soilcat_id': nan,
 'function_type': 'Altres',
 'category_type': nan,
 'fluid_type': 'UNITARIES',
 'location_type': nan,
 'workcat_id': nan,
 'workcat_id_end': nan,
 'buildercat_id': nan,
 'builtdate': nan,
 'enddate': nan,
 'ownercat_id': 'ALTRES',
 'muni_id': 1,
 'postcode': nan,
 'streetaxis_id': nan,
 'postnumber': nan,
 'postcomplement': nan,
 'streetaxis2_id': nan,
 'postnumber2': nan,
 'postcomplement2': nan,
 'descri

Comprovem quants nodes tenen valors nulls a top_elev o un top_elev>300.
Una vegada els tenim, corretgim les altures amb el valor del seu successor.
Primer, posem tots els valors que creiem que estan be de top_elev a costum_top_elev

In [88]:
import math

nodes_erronis = []

for node, data in subGraph.nodes(data=True):
    if not math.isnan(data['top_elev']) and not data['top_elev']>300:
        data['custom_top_elev'] = data['top_elev']
    else: 
        data['custom_top_elev'] = math.nan
        nodes_erronis.append(node)
print('nodes_erronis incials', len(nodes_erronis))


i = 0
while len(nodes_erronis) > 0:
    tmp = []
    for node_erroni in nodes_erronis:
        data_err = subGraph.nodes[node_erroni]
        neighbors = list(nx.all_neighbors(subGraph, node_erroni))
        costums_top_elev_neighbors = [data['custom_top_elev'] for node, data in subGraph.nodes(data=True) if node in neighbors and not math.isnan(data['custom_top_elev'])]
        if len(costums_top_elev_neighbors) > 0:
            data_err['custom_top_elev'] = np.mean(costums_top_elev_neighbors)
        else:
            tmp.append(node_erroni)
    i+=1
    nodes_erronis = tmp.copy()
    print('nodes_erronis a iteracio', i, ': ', len(nodes_erronis))

nodes_erronis incials 4382
nodes_erronis a iteracio 1 :  1624
nodes_erronis a iteracio 2 :  886
nodes_erronis a iteracio 3 :  489
nodes_erronis a iteracio 4 :  277
nodes_erronis a iteracio 5 :  153
nodes_erronis a iteracio 6 :  90
nodes_erronis a iteracio 7 :  54
nodes_erronis a iteracio 8 :  32
nodes_erronis a iteracio 9 :  14
nodes_erronis a iteracio 10 :  4
nodes_erronis a iteracio 11 :  1
nodes_erronis a iteracio 12 :  0
12


In [91]:
nodes_top_elev_erronis = [node for node, data in subGraph.nodes(data=True) if math.isnan(data['ymax']) or data['ymax']<0.25]
len(nodes_top_elev_erronis)

2958

### 3.1 GENERACIÓ DEL REPORT

Hi ha errors amb el code de molts nodes...

In [179]:
f = open("results/reports/AdG.txt", "w")
f.write("El llistat de nodes que no arriben a la depuradora és el següent:\n")
for node in malament:
  id = subGraph.nodes[node]['code']
  if id[0:2] == 'PR':
    if node != malament[-1]:
      f.write(str(subGraph.nodes[node]['code'])+", ")
    else:
      f.write(str(subGraph.nodes[node]['code'])+".")
  else:
    if node != malament[-1]:
      f.write(str(subGraph.nodes[node]['node_id'])+", ")
    else:
      f.write(str(subGraph.nodes[node]['node_id'])+".")
f.write("\n\nPer arreglar-los s'han canviat les direccions dels arcs següents:\n")
for arc in arcs_girats:
  if arc != arcs_girats[-1]:
    f.write(str(arc)+", ")
  else:
    f.write(str(arc)+".")
f.close()


## 4. S'ELIMINEN ELS OUTFALLS I ELS NODES QUE ARRIBEN A ELLS DEL SUBGRAF

Posar threshold en els nodes de sobreiximent per construir xarxa en sec


In [180]:
#volem extreure els node outfall de sobreiximent i els seus predecesors del subGrap
delete = [node for node, data in subGraph.nodes(data=True) if not nx.has_path(subGraph, node, edar)]
subGraph.remove_nodes_from(delete)
# nota: estem extraient els nodes direccionats com a desembocament del subGrap exceptuant la depuradora hauria de ser el mateixos que els marcats com a outfall

# Si tornem a veure els nodes que no arriben a la depuradora, no haurien de quedar cap
no_llegan_depuradora_arreglat = [node for node in subGraph.nodes if not nx.has_path(subGraph, node, edar)]
        
print('nodes que no arriben a la depuradora després d\'eliminar els nodes de sobreiximent =', len(no_llegan_depuradora_arreglat))

print('La subxarxa en que tots els nodes arriben a la depuradora conte: ')
print('nodes = ', len(subGraph.nodes))
print('arcs =', len(subGraph.edges))

nodes que no arriben a la depuradora després d'eliminar els nodes de sobreiximent = 0
La subxarxa en que tots els nodes arriben a la depuradora conte: 
nodes =  11196
arcs = 11680


## 5. IMPRIMIR EL SUBGRAF CONNECTAT A LA DEPURADORA

In [181]:
#imprimim el graph en un .gpkg per visualitzar en QGIS

#transformem les coordenades
transformer = Transformer.from_crs("EPSG:25831", "WGS84")

#afegim els nodes
for index, row in subGraph.nodes(data=True):
    if row['state'] != 0:
        x = row['geometry'].x
        y = row['geometry'].y

        long, lat = transformer.transform(x, y)
        row['longitude'] = long
        row['latitude'] = lat
        row['x'] = x
        row['y'] = y
        
        row['geometry'] = (long, lat)    

# fem una copia del subgraph i el reindexem amb la geometria dels nodes
subGraph_copy = subGraph.copy()
geometry_nodes = {node: data['geometry'] for node, data in subGraph_copy.nodes(data=True)}
subGraph_copy = nx.relabel_nodes(subGraph_copy, geometry_nodes)

# guardem el subgraph en dos .gpkg, un per nodes i un per arcs
points, lines = mm.nx_to_gdf(subGraph_copy)
points.set_geometry("geometry")
points.to_file("results/networks/nodes_corretgits.gpkg", layer='nodes', driver="GPKG")
lines.set_geometry("geometry")
lines.to_file("results/networks/arcs_corretgits.gpkg", layer='arcs', driver="GPKG")

/home/sergi/.local/lib/python3.10/site-packages/momepy/utils.py:430: UserWarning: Approach is not set. Defaulting to 'primal'.
  warnings.warn("Approach is not set. Defaulting to 'primal'.")


# 6. correccio links

1. Els nodes y arcs del subGraph tenen links associats que porten a guilly y connec (features_type -> defineix la llista on es troba el feature, feature_id -> correspon a connec/guilly_id). volem seleccionar nomes aquells elements que pertanyen al nostre subGraph. els links estan associat a un node/edge del nostre subGraph per geometry y exit_id (exit_id -> arc/node_id del nostre subGraph). 

      partim de tres llistes:
                                                                      
                                                                      --  feautre_type -> connec/guilly (in list) 
            link [link_id, feature_type, feature_id, exit_id, row]    --  feature_id -> coonec/guilly_id
                                                                      --  exit_id -> node/arc_id  (subGraph)

            connec [connec_id, link_id, row]

            guilly [guilly_id, link_id, row]

      hem de crear tres noves sub-llistes dels elements connectats al nostre SubGrap.

            edges_link

            nodes_connec

            nodes_guilly  

NOTA: connec son els desembocaments de les cases y edificis a la xarxa (conexions entre xarca interna de cada edifici y xarxa externa de la ciutat), crec que no tots han de ser sanitaris, alguns poden ser reculls de aigua pluvial del terrat (preguntar a nicole)

NOTA: guilly son els embornals del carrer (arquetes) que recullen nomes (?) aigues pluvials.


2. Per mantenir una xarxa topologicament consistent, un edge (link) no pot anar associat a un altre edge (arc) com node_2, caldria crear un altre node en la interseccio de l'arc que no suposaria cap benefici y augmentaria la complexitat del subGraph. farem el seguent:

      els nostres nodes_connec, nodes_guillu han destar associats al SubGrap a traves de un edges_link direccionat a un node de la xarxa, no a un arc.

      els edges_link que estan direccionats a un arc de la xarxa aniran redireccionats a un dels nodes asociats del corresponent arc (node_1 / node_2)
                  
      nota: No importa gaire si el edge_link va a node_1 o node_2 pero es podria fer per proximitat de geometry.




# 7. correccio mesures 

study geometry: Chamber, Outfall, Manhole, Juntion 

in nodes and arcs, check None's

 -top_elev 

 -ymax

 -negative slope

 -afegir bombes

aproximar errors en mesures: volums, mesures, geometries -> autocalibratge ?

Canviar el tipus de node de la depuradora a outfall en el subGrap

## REVISAR A PARTIR D'AQUÍ

2. extreure els nodes/branca de sobreiximent per construir xarxa en sec

extreure tots els node_type=outfall i les seves branques precedents

In [ ]:
#volem extreure els node outfall de sobreiximent i els seus predecesors del subGrap
delete = [node for node, data in subGraph.nodes(data=True) if subGraph.out_degree(node) == 0 and node!=edar]
while len(delete) != 0:
    G.remove_nodes_from(delete)
    delete = [node for node, data in subGraph.nodes(data=True) if subGraph.out_degree(node) == 0 and node!=edar]
# nota: estem extraient els nodes direccionats com a desembocament del subGrap exceptuant la depuradora hauria de ser el mateix que els marcats com a outfall

3. check geometry: mesures in nodes and arcs, check Null's

 -top_elev 
 -ymax
 -slope
 -bombes

 

4. Selexionem nomes aquells Link entre el subGraph i els Connc




In [194]:
# EXTRACT ORPHAN NODE
node = gpd.read_file("node.gpkg")
node.head(2)  # Prints the first 5 rows of the loaded data to see what it looks like.
print(node.shape)
node_orphan = gpd.read_file("node_orphan.gpkg")
node_orphan.head(2)  # Prints the first 5 rows of the loaded data to see what it looks like.
print(node_orphan.shape)
difference = set(node['node_id']) - set(node_orphan['id'])
node_not_orphan = node[node['node_id'].isin(list(difference))]
print(node_not_orphan.shape)


# EXTRACT ORPHAN ARC
arc = gpd.read_file("arc.gpkg")
arc.head(2)  # Prints the first 5 rows of the loaded data to see what it looks like.
print(arc.shape)
arc_orphan = gpd.read_file("arc_orphan.gpkg")
arc_orphan.head(2)  # Prints the first 5 rows of the loaded data to see what it looks like.
print(arc_orphan.shape)
difference = set(arc['arc_id']) - set(arc_orphan['id'])
arc_not_orphan = arc[arc['arc_id'].isin(list(difference))]
print(arc_not_orphan.shape)


# EXTRACT ORPHAN LINK

#ERROR: must be done with link.gpkg extracted fom giswater project
link = gpd.read_file("link.gpkg")
print(link.shape)
link_orphan = link[link['exit_id'].isin(list(set(node_orphan['id']) and set(arc_orphan['id'])))]
print(link_orphan.shape)
difference = set(link['link_id']) - set(link_orphan['link_id'])
link_not_orphan=link[link['link_id'].isin(list(difference))]
print(link_not_orphan.shape)

(13572, 73)
(172, 3)
(13400, 73)
(13722, 88)
(171, 3)
(13551, 88)
(23474, 18)
(31, 18)
(23443, 18)
